# Regressionanalyse Wiedergabedauer

In [ ]:

# Zelle 1: Setup und Imports
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Plotstil festlegen (verwenden einen basic matplotlib style)
plt.style.use('default')
# Seaborn Plotting Style setzen
sns.set_theme(style="whitegrid")

%matplotlib inline

plt.rcParams['font.family'] = 'DejaVu Sans'  # Alternativ: 'Liberation Sans', 'Arial Unicode MS'

# Display-Optionen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Zelle 2: Laden der vorverarbeiteten Daten
# Laden der Pickle-Datei
with open('../data/processed/processed_data.pkl', 'rb') as f:
    data = pickle.load(f)

# Extrahieren der einzelnen Komponenten
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']
scaler = data['scaler']
feature_columns = data['feature_columns']
target_columns = data['target_columns']

# Überprüfung der geladenen Daten
print("Daten erfolgreich geladen!")
print(f"\nTrainings-Set Dimensionen:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"\nFeature-Spalten:")
print(feature_columns)
print(f"\nZiel-Spalten:")
print(target_columns)

# Zelle 3: Kurze Dateninspektion
# Überblick über die Feature-Verteilungen
print("\nStatistische Kennzahlen der Features im Trainingsset:")
print(X_train.describe())

# Überblick über die Zielvariblen
print("\nStatistische Kennzahlen der Zielvariablen im Trainingsset:")
print(y_train.describe())

# Visualisierung der Verteilung der Zielvariablen
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Verteilung der Zielvariablen')

for i, col in enumerate(target_columns):
    sns.histplot(data=y_train, x=col, ax=axes[i])
    axes[i].set_title(col)
    
plt.tight_layout()
plt.show()

#### Allgemeine Analyse

In [12]:
def analyze_features_for_target(X_train, y_train, target_name, feature_columns):
    """
    Führt eine umfassende Feature-Analyse für eine Zielvariable durch
    """
    from sklearn.feature_selection import SelectKBest, f_regression
    from sklearn.linear_model import Lasso
    from sklearn.impute import SimpleImputer
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    # Nur numerische Spalten auswählen
    numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
    X_numeric = X_train[numeric_features]
    
    # Imputation der fehlenden Werte
    imputer = SimpleImputer(strategy='mean')
    X_numeric_imputed = pd.DataFrame(
        imputer.fit_transform(X_numeric),
        columns=X_numeric.columns
    )
    
    target = y_train[target_name]
    results = {}
    
    # 1. F-Regression
    selector = SelectKBest(score_func=f_regression, k='all')
    selector.fit(X_numeric_imputed, target)
    
    f_scores = pd.DataFrame({
        'Feature': numeric_features,
        'F_Score': selector.scores_,
        'P_Value': selector.pvalues_
    }).sort_values('F_Score', ascending=False)
    
    results['f_scores'] = f_scores
    
    # 2. Lasso
    lasso = Lasso(alpha=0.01)
    lasso.fit(X_numeric_imputed, target)
    
    lasso_importance = pd.DataFrame({
        'Feature': numeric_features,
        'Coefficient': np.abs(lasso.coef_)
    }).sort_values('Coefficient', ascending=False)
    
    results['lasso'] = lasso_importance
    
    # 3. Korrelationsanalyse
    correlation_matrix = X_numeric_imputed.corr()
    high_correlations = np.where(np.abs(correlation_matrix) > 0.7)
    high_corr_pairs = [(correlation_matrix.index[x], correlation_matrix.columns[y], 
                        correlation_matrix.iloc[x, y])
                       for x, y in zip(*high_correlations) 
                       if x != y and x < y]
    
    results['high_correlations'] = high_corr_pairs
    
    # Visualisierungen
    plt.figure(figsize=(12, 6))
    sns.barplot(data=f_scores.head(10), x='F_Score', y='Feature')
    plt.title(f'Top 10 Features nach F-Score für {target_name}')
    plt.tight_layout()
    plt.show()
    
    # Zusätzliche Information über fehlende Werte
    missing_values = X_numeric.isnull().sum()
    results['missing_values'] = missing_values[missing_values > 0]
    
    return results
# Verwendung:
# Führen Sie diese Analyse für jede Zielvariable durch:
# results = analyze_features_for_target(X_train, y_train, 'durchschnittliche_wiedergabedauer', feature_columns)
# 
# print("Top 10 Features nach F-Score:")
# print(results['f_scores'].head(10))
# 
# print("\nTop 10 Features nach Lasso-Koeffizienten:")
# print(results['lasso'].head(10))
# 
# print("\nStark korrelierte Features:")
# for pair in results['high_correlations']:
#     print(f"{pair[0]} -- {pair[1]}: {pair[2]:.3f}")

# Jetzt folgen F-Test-Analyse und Korrelationen zur Zielvariable Wiedergabedauer

In [ ]:
# Analyse für durchschnittliche Wiedergabedauer
results = analyze_features_for_target(X_train, y_train, 'durchschnittliche_wiedergabedauer', feature_columns)

# Ausgabe der Ergebnisse
print("Top 10 Features nach F-Score:")
print(results['f_scores'].head(10))

print("\nTop 10 Features nach Lasso-Koeffizienten:")
print(results['lasso'].head(10))

print("\nStark korrelierte Features:")
for pair in results['high_correlations']:
    print(f"{pair[0]} -- {pair[1]}: {pair[2]:.3f}")

print("\nFehlende Werte in den Features:")
print(results['missing_values'])

Hier eine Analyse, ohne Live-Videos.

In [ ]:
# Daten ohne Live-Content filtern
X_train_filtered = X_train[X_train['theme_Live'] == 0].copy()
y_train_filtered = y_train.loc[X_train_filtered.index]

print("Ursprüngliche Datengröße:", len(X_train))
print("Gefilterte Datengröße:", len(X_train_filtered))

# Feature-Analyse mit gefilterten Daten
results = analyze_features_for_target(X_train_filtered, y_train_filtered, 'durchschnittliche_wiedergabedauer', feature_columns)

# Ausgabe der Ergebnisse
print("\nTop 10 Features nach F-Score:")
print(results['f_scores'].head(10))

print("\nTop 10 Features nach Lasso-Koeffizienten:")
print(results['lasso'].head(10))

print("\nStark korrelierte Features:")
for pair in results['high_correlations']:
    print(f"{pair[0]} -- {pair[1]}: {pair[2]:.3f}")

# Zusätzlich: Korrelation der Titelbewertung mit Wiedergabedauer für gefilterte Daten
correlation = X_train_filtered['Bewertung_Titel'].corr(y_train_filtered['durchschnittliche_wiedergabedauer'])
print("\nKorrelation Titelbewertung mit Wiedergabedauer (ohne Live):", correlation)

Diese neuen Ergebnisse ohne Live-Content zeigen einige interessante Veränderungen:

Hauptunterschiede zu vorher:

Der F-Score für video_length_seconds ist deutlich höher (4170 vs. 390 vorher)
Die Bedeutung von verlorene_abonnenten und gewonnene_abonnenten ist stark gesunken
Audience-Metriken sind wichtiger geworden
Der Lasso-Koeffizient für video_length_seconds ist viel größer (2.127 vs. 0.417 vorher)


Top-Prädiktoren nach beiden Methoden:

video_length_seconds (mit sehr großem Abstand der wichtigste Prädiktor)
wiedergabezeit_minuten
Gestaltung_Thumbnail
geteilte_inhalte
likes


Titelbewertung:

Die Korrelation ist jetzt sogar leicht negativ (-0.121)
Dies bestätigt, dass die Titelbewertung keinen positiven Einfluss auf die Wiedergabedauer hat



Empfehlung für das Regressionsmodell:

Hauptprädiktoren:

video_length_seconds als wichtigster Prädiktor
Gestaltung_Thumbnail
likes oder geteilte_inhalte (aber nicht beide wegen möglicher Kollinearität)


Optionale Prädiktoren:

Eine ausgewählte Audience-Metrik (z.B. audience_female_age25-34)
NICHT die Titelbewertung aufgrund der negativen Korrelation



Die Ergebnisse sind jetzt viel klarer und zeigen, dass Live-Content tatsächlich das Bild verzerrt hat. Möchten Sie, dass wir mit diesen bereinigten Daten ein Regressionsmodell erstellen?

Hier eine Analyse, inwieweit die Bewertung des Titels eine Rolle spielt. 

In [ ]:
# Analyse der Titelbewertung
print("Statistiken zur Titelbewertung:")
print(X_train['Bewertung_Titel'].describe())

# Korrelation mit der Zielvariable
correlation = X_train['Bewertung_Titel'].corr(y_train['durchschnittliche_wiedergabedauer'])
print("\nKorrelation mit durchschnittlicher Wiedergabedauer:", correlation)

# Visualisierung
plt.figure(figsize=(10, 6))
plt.scatter(X_train['Bewertung_Titel'], y_train['durchschnittliche_wiedergabedauer'], alpha=0.5)
plt.xlabel('Bewertung Titel')
plt.ylabel('Durchschnittliche Wiedergabedauer')
plt.title('Zusammenhang zwischen Titelbewertung und Wiedergabedauer')
plt.show()

# Prüfen auf fehlende Werte
missing = X_train['Bewertung_Titel'].isnull().sum()
print("\nAnzahl fehlender Werte in Bewertung_Titel:", missing)

In [ ]:
# Zielvariable definieren
target_variable = 'aufrufe'

# Merkmale definieren
categorical_features = [col for col in X_train.columns if X_train[col].dtype == 'bool' or X_train[col].dtype == 'object']
numerical_features = [col for col in X_train.columns if X_train[col].dtype in ['float64', 'int64']]

# Zusammenführen der Zielvariable und der Prädiktoren
train_data = X_train.copy()
train_data[target_variable] = y_train[target_variable]

# Boxplots für kategoriale Merkmale
for feature in categorical_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=train_data[feature], y=train_data[target_variable])
    plt.title(f'Boxplot: {target_variable} nach {feature}')
    plt.ylabel('Aufrufe')
    plt.xlabel(feature)
    plt.xticks(rotation=30)
    plt.show()

# Streudiagramme für numerische Merkmale
for feature in numerical_features:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=train_data[feature], y=train_data[target_variable])
    plt.title(f'Streudiagramm: {target_variable} vs. {feature}')
    plt.ylabel('Aufrufe')
    plt.xlabel(feature)
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Daten für Streudiagramm vorbereiten
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x=feature_columns['video_length_seconds'],  # Videolänge in Sekunden
    y=target_columns['aufrufe'],  # Zielvariable: Aufrufe
    hue=feature_columns['Thema'],  # Farbige Gruppierung nach Thema
    palette='Set2',  # Farbschema für Übersichtlichkeit
    alpha=0.7  # Transparenz, um Überlagerungen zu reduzieren
)

# Titel und Achsenbeschriftungen hinzufügen
plt.title('Streudiagramm: Aufrufe vs. Videolänge (nach Thema gruppiert)')
plt.xlabel('Videolänge (Sekunden)')
plt.ylabel('Aufrufe')
plt.legend(title='Thema', bbox_to_anchor=(1.05, 1), loc='upper left')  # Legende rechts neben dem Plot
plt.tight_layout()
plt.show()

In [ ]:
sns.histplot(X_train['Bewertung_Titel'])
plt.title('Verteilung der Bewertung_Titel')
plt.show()
sns.histplot(X_train['Gestaltung_Thumbnail'])
plt.title('Verteilung der Gestaltung_Thumbnail')
plt.show()